# Inflation Impact Analysis: Overview

Inflation occurs when there is a sustained increase in the general price level of goods and services in an economy over time.It impacts various aspects of the economy, including purchasing power, consuming behavior, savings and investment. Moderate inflation is typically a sign of healthy, growing economy, as it encourages spending and investment. However, high or unpredictable inflation can erode the value of money, disrupt financial planning, and lead to economic uncertainity.

To analyze the impact of inflation, we need to compare it with other economic indicators. So, to analyze the impact of inflation on the economy, we will compare it with exchange rates over time. This comparison is important because exchange rates are influenced by inflation differentials between countries, such that higher inflation in a country generally leads to weaker currency relative to countries with lower inflation.

In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

inflation_data = pd.read_csv('../data/Inflation_Rates_Transformed-1.csv')
exchange_rate_data = pd.read_csv('../data/USD_INR_Exchange_Rates_1980_2024.csv')

exchange_rate_data.head()

,Year,Average Exchange Rate (INR/USD)
0,2003,45.476087
1,2004,45.188604
2,2005,44.003753
3,2006,45.183561
4,2007,41.192248


In [4]:
inflation_data.head()

,Country,Year,Inflation Rate
0,India,1980,11.3
1,United States,1980,13.5
2,Emerging market and developing economies,1980,26.0
3,India,1981,12.7
4,United States,1981,10.4


I'll now prepare this dataset for analyais by merging the relevants parts of these datasets to analyze the impact of inflation on exchange rates:

In [1]:
%pip install -U kaleido

   ---------------------------------------- 0.0/65.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/65.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/65.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/65.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/65.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/65.9 MB ? eta -:--:--
   ---------------------------------------- 0.5/65.9 MB 620.6 kB/s eta 0:01:46
    --------------------------------------- 1.0/65.9 MB 1.2 MB/s eta 0:00:57
   - -------------------------------------- 1.8/65.9 MB 1.8 MB/s eta 0:00:36
   - -------------------------------------- 2.4/65.9 MB 2.1 MB/s eta 0:00:30
   -- ------------------------------------- 3.7/65.9 MB 2.6 MB/s eta 0:00:25
   -- ------------------------------------- 4.2/65.9 MB 2.5 MB/s eta 0:00:25
   --- ------------------------------------ 5.0/65.9 MB 2.8 MB/s eta 0:00:22
   --- --------------------------------

In [5]:
# filter inflation data for India and the United States
inflation_filtered_df = inflation_data[inflation_data['Country'].isin(['India', 'United States'])]

# pivot the inflation data to have separate columns for each country inflation rates
inflation_pivot_df = inflation_filtered_df.pivot(index='Year', columns='Country', values='Inflation Rate').reset_index()

# merge the exchange rate data with the inflation data
merged_df = pd.merge(exchange_rate_data, inflation_pivot_df, on='Year')

# renaming columns
merged_df.columns = ['Year', 'Exchange Rate (INR/USD)', 'Inflation Rate(India)', 'Inflation Rate(United States)']

merged_df.head()

,Year,Exchange Rate (INR/USD),Inflation Rate(India),Inflation Rate(United States)
0,2003,45.476087,4.1,2.3
1,2004,45.188604,4.0,2.7
2,2005,44.003753,3.7,3.4
3,2006,45.183561,6.7,3.2
4,2007,41.192248,5.9,2.9


The merged dataset now contains the following columns for each year:
    **Exchange Rate (INR/USD):** The average exchange rate of INR to USD
    **Inflation Rate(India)**: The inflation rate for India
    **Inflation Rate(United States)**: The inflation rate for United States

Let's start by analyzing the trend of inflation rates for both India and United States alongside the exchange rate:

In [6]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=3, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.1,
                    subplot_titles=("Trend of Exchange Rate (INR/USD)",
                                    "Trend of Inflation Rate (India)",
                                    "Trend of Inflation Rate (United States)"))

fig.add_trace(go.Scatter(x=merged_df['Year'],
                         y=merged_df['Exchange Rate (INR/USD)'],
                         mode='lines+markers',
                         marker=dict(color='blue'),
                         name='Exchange Rate (INR/USD)'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=merged_df['Year'],
                         y=merged_df['Inflation Rate(India)'],
                         mode='lines+markers',
                         marker=dict(color='orange'),
                         name='Inflation Rate (India)'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=merged_df['Year'],
                         y=merged_df['Inflation Rate(United States)'],
                         mode='lines+markers',
                         marker=dict(color='green'),
                         name='Inflation Rate (United States)'),
              row=3, col=1)

fig.update_layout(height=800,
                  width=900,
                  showlegend=False,
                  title_text="Trends of Exchange Rate and Inflation Rates",
                  xaxis3_title="Year",
                  template='plotly_white')

fig.update_yaxes(title_text="Exchange Rate (INR/USD)", row=1, col=1)
fig.update_yaxes(title_text="Inflation Rate (%)", row=2, col=1)
fig.update_yaxes(title_text="Inflation Rate (%)", row=3, col=1)

fig.show()

fig.write_image("../image/Trends of Exchange Rate and Inflation Rates.png")

The exchange rate shows a general upward trend over these years, which indicates a depreciation of the Indian Rupee against US Dollar. However, there are periods of both sharp increases and realtive stability.

India's inflation rate has flactuated signficantly over the years, with periods of high inflation (e.g. early 2000s) and more stable inflation in recent years. The United States has generally exprienced lower and more stable inflation compared to India, with fewer extreme flactuations.

Next, let's perform a correlation analysis to explore the realtionship between inflation rates and the exchange rates:

In [6]:
correlation_matrix = merged_df[['Exchange Rate (INR/USD)', 'Inflation Rate(India)', 'Inflation Rate(United States)']].corr()
correlation_matrix

,Exchange Rate (INR/USD),Inflation Rate(India),Inflation Rate(United States)
Exchange Rate (INR/USD),1.000000,-0.337748,0.235321
Inflation Rate(India),-0.337748,1.000000,-0.120503
Inflation Rate(United States),0.235321,-0.120503,1.000000


Findings from the comparitive analysis:

1.**Early 2000s**: A period of high inflation in India coincides with a period of relative stability in the exchange rate. It suggests that factors other than inflation may have been driving the exchange rate during this time.
2.**Late 2000s to Early 2010s**: The period shows some alignment between rising inflation in India and a weakening INR, which indicates that inflation could be contributing to exchange rate movements.
3.**2015 Onwards**: The exchange rate continues to rise, while both India’s and the United States’ inflation rates remain relatively low. This divergence suggests that the exchange rate is influenced by additional factors beyond inflation, such as economic growth, monetary policy, and international trade dynamics.

# Analyzing Inflation based on the Purchasing Power Parity (PPP)

Purchasing Power Parity (PPP) is an economic theory that suggests that in the long term, exchange rates between two countries should adjust so that a basket of goods costs the same in both countries when priced in a common currency. PPP is used as a method to compare the economic productivity and standards of living between different countries. If one country’s inflation rate is higher than another’s, its currency should depreciate accordingly to maintain parity in purchasing power to ensure that the same goods cost the same in both locations.

We will now analyze whether the Purchasing Power Parity theory holds by comparing the relative inflation rates and exchange rate movements over time. It will provide a deeper understanding of whether the exchange rate aligns with the theoretical value based on inflation differentials.

To test whether PPP holds for India and the United States, we can:

1. Calculate the Expected Rate Based on PPP
2. Compare the Actual Exchange Rate with the PPP-Based Exchange Rate

The formula for PPP-based Exchange rate predcition is:

**Expected Exchange Rate = Initial Exchange Rate × ( 1 + Inflation Rate in India / 1 + Inflation Rate in the US)**

Let’s calculate and visualize the PPP-based expected exchange rate versus the actual exchange rate:

In [7]:
initial_exchange_rate = merged_df['Exchange Rate (INR/USD)'].iloc[0]

# calculate expected exchange rate based on PPP
merged_df['Expected Exchange Rate (PPP)'] = initial_exchange_rate * (
    (1 + merged_df['Inflation Rate(India)'] / 100) / (1 + merged_df['Inflation Rate(United States)'] / 100)
).cumprod()

fig = go.Figure()

# plot actual exchange rate
fig.add_trace(go.Scatter(x=merged_df['Year'],
                         y=merged_df['Exchange Rate (INR/USD)'],
                         mode='lines+markers',
                         name='Actual Exchange Rate (INR/USD)',
                         line=dict(color='blue')))

# plot PPP-based expected exchange rate
fig.add_trace(go.Scatter(x=merged_df['Year'],
                         y=merged_df['Expected Exchange Rate (PPP)'],
                         mode='lines+markers',
                         name='Expected Exchange Rate (PPP)',
                         line=dict(color='orange', dash='dash')))

fig.update_layout(title='Actual vs. Expected Exchange Rate (PPP)',
                  xaxis_title='Year',
                  yaxis_title='Exchange Rate (INR/USD)',
                  legend_title_text='Exchange Rates',
                  template='plotly_white',
                  height=600,
                  width=1000)

fig.show()

fig.write_image("../image/Actual vs. Expected Rate.png")

The blue line represents the actual exchange rate (INR/USD) over time, while the orange dashed line represents the exchange rates based on PPP. In some periods, the actual exchange rate closely follows the expected PPP-based rate, which suggests that PPP holds. However, in other periods, there are significant deviations between the two.

The PPP-based expected exchange rate shows a more rapid increase compared to the actual exchange rate. It suggests that, according to PPP, the INR should have depreciated more than it actually did. However, the actual exchange rate was consistently lower than the PPP-based expected rate, which indicates that factors other than inflation are at play.

# Conclusion

Our analysis revealed that inflation in India and the United States influences the exchange rate between INR and USD. Higher inflation in India generally leads to a depreciation of the INR relative to the USD, while lower inflation in the United States contributes to a stronger USD. While inflation affects the exchange rate between INR and USD, it is only one of many factors.